## 2. 강화학습 문제의 모형화

<div style="text-align: right"> <b>Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial issue : 2023.06.29 </div>
<div style="text-align: right"> last update : 2023.06.29 </div>

In [ ]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import warnings; warnings.filterwarnings('ignore')
#plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

# Options for pandas
pd.options.display.max_columns = 200
pd.options.display.max_rows = 100

### 2.2 one-armed bandit, 슬롯머신 문제

한 시행에서 에이전트는 n개의 슬로머신 중 하나를 선택해서 레버를 당긴다.   
한 시행에서 에이전트가 할수 있는 총 동작은 n가지이다.   
이러한 시행을 k번 반복한다.
k번째 시행의 액션 a로 부터 보상 $R_k$f를 받는다.

이 문제를 푸는 전략은 처음 몇번동안 서로 다른 레버를 당겨서 리워드를 관찰하는것이다.  
그 다음 관찰된 보상이 가장 큰 레버를 계속 당기는 것이다.   

이 전략을 위해서 이전 시행들에 기초한 특정 액션의 기대 보상, expected reward라는 개념이 필요하다. k번째 시행에서 액션 a의 기대 보상을 수식으로 $Q_k(a)$라고 표기한다.   
즉 기대 보상은 액션의 함수이며, 이 함수는 액션을 주어진 액션을 취했을 때 기대할 수있는 리워드의 양을 돌려준다.

In [2]:
def exp_reward(action, history):
    rewards_for_action = history[action]
    return sum(rewards_for_action) / len(rewards_for_action)

- 즉 k번째 시행에서 액션 a의 기대 보상은 그때까지 a의 모든 리워드의 산술 평균이다.   
- 따라서 이전 액션들과 관찰은 이후 액션에 영향을 미친다. 이를 이전 동작들이 현재 동작과 미래 동작들을 강화한다고 말할 수 있다.  
- $Q_k(a)$를 뭔가 가치를 말해준다는 점에서 동작 가치함수(action value function)라고 부른다.

기대 보상에 근거해서 최선의 동작을 찾는 코드

In [3]:
def get_best_action(actions):
    best_action = 0
    max_action_value = 0
    for i in range(len(actions)):
        cur_action_value = get_action_value(actions[i])
        if cur_action_value > max_action_value:
            best_action = i
            max_action_value = cur_action_value
            
    return best_action

- 위 방식을 사용하면 아직 시도한 적없는 어쩌면 기대 보상이 더 큰 레버를 선택하는 일은 없다. 
- 이처럼 지금까지의 경험에만 근거해 최선의 것을 선택하는 접근 방식을 greedy method라고 한다.